In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pickle
import traceback
from multiprocessing import Pool
import os
import random
from functools import partial


def find_genre(html_str):
    pattern = '<div class="section"> <a href="https://apkplz.net"> Home </a> » <a href="https://apkplz.net/category/'
    id_start = html_str.index(pattern) + len(pattern)
    html_str = html_str[id_start:]
    id_start = html_str.index('>') + 1
    id_end = html_str[id_start:].index('</a>')
    genre = html_str[id_start:id_start + id_end]
    return genre


def find_name(html_str):
    pattern = '<div class="col-sm-12 col-md-8"> <h1>'
    id_start = html_str.index(pattern) + len(pattern)
    id_end = html_str[id_start:].index('</h1>')
    name = html_str[id_start:id_start + id_end]
    return name


def find_requierments(html_str):
    pattern = '<li>Requires:'
    id_start = html_str.index(pattern) + len(pattern)
    id_end = html_str[id_start:].index('</li>')
    requierments = html_str[id_start:id_start + id_end]
    return requierments


def find_price(html_str):
    pattern = '<li> Price:'
    id_start = html_str.index(pattern) + len(pattern)
    id_end = html_str[id_start:].index('</li>')
    price = html_str[id_start:id_start + id_end]
    return price


def find_fileSize(html_str):
    pattern = '<li>File size:'
    id_start = html_str.index(pattern) + len(pattern)
    id_end = html_str[id_start:].index('</li>')
    fileSize = html_str[id_start:id_start + id_end]
    return fileSize[:-2]


def find_rate(html_str):
    pattern = '<li>Rate'
    id_start = html_str.index(pattern) + len(pattern)
    id_end = html_str[id_start:].index('stars')
    rate = float(html_str[id_start:id_start + id_end])

    html_str = html_str[id_start:]
    pattern = 'based on'
    id_start = html_str.index(pattern) + len(pattern)
    id_end = html_str[id_start:].index('reviews')
    rewiew = int(html_str[id_start:id_start + id_end])
    if rewiew > 100:
        return rate
    else:
        return 0


def find_description(html_str):
    pattern = "please contact us."
    id_start = html_str.index(pattern) + len(pattern)
    id_end = html_str[id_start:].index('</div>')
    description = html_str[id_start:id_start + id_end]
    return description




def scrap(android_bundles):
    url_google_play = 'https://play.google.com/store/apps/details?id='
    url_apkplz = "https://apkplz.net/app/"
    url_q_apkplz = "https://apkplz.net/search?q="
    bad_bundles = []
    features = {}

    counter = 0
    save_id = 0

    while counter < len(android_bundles):
        current_bundle = android_bundles[counter]
        # if counter == 100:
        #     break
        # if counter % 5 == 0:
        #     with open('features'+str(save_id)+'.pickle', 'wb') as f:
        #         pickle.dump(features, f)
        #     with open('bad'+str(save_id)+'.pickle', 'wb') as f:
        #         pickle.dump(bad_bundles, f)
        #     save_id = save_id + 1
        #     bad_bundles = []
        #     features = {}
        #
        try:
#             time.sleep(2)
            #         current_bundle = "com.KRAKET.Milohin"

            current_app_apkplz_url = url_q_apkplz + current_bundle
            response = requests.get(current_app_apkplz_url, timeout=10)

            current_app_apkplz_url = url_apkplz + current_bundle
            response = requests.get(current_app_apkplz_url, timeout=10)
            soup = BeautifulSoup(response.text, "html.parser")
            str_soup = str(soup)

            try:
                name = find_name(str_soup)
            except:
                name = np.nan
            try:
                price = find_price(str_soup)
            except:
                price = np.nan
            try:
                requierments = find_requierments(str_soup)
            except:
                requierments = np.nan
            try:
                fileSize = find_fileSize(str_soup)
            except:
                fileSize = np.nan
            try:
                rate = find_rate(str_soup)
            except:
                rate = np.nan
            try:
                description = find_description(str_soup)
            except:
                description = np.nan

            features[current_bundle] = {}
            features[current_bundle]["name"] = name
            features[current_bundle]["price"] = price
            features[current_bundle]["fileSize"] = fileSize
            features[current_bundle]["rate"] = rate
            features[current_bundle]["description"] = description
            features[current_bundle]["requierments"] = requierments
            features[current_bundle]["genre"] = find_genre(str_soup)


            print("good", current_bundle, counter / len(android_bundles))
            counter = counter + 1
        except Exception as e:
            counter = counter + 1
            # time.sleep(5)
            print("bad", current_bundle, counter / len(android_bundles))
            print(e)
            bad_bundles.append(current_bundle)
            continue
    return features, bad_bundles


if __name__ == "__main__":

    with open('bundles.pickle', 'rb') as f:
        bundles = pickle.load(f)

    ios_bundles = list()
    android_bundles = list()

    for i in bundles:
        try:
            tmp = int(i)
            ios_bundles.append(i)
        except:
            android_bundles.append(i)
    drop_nan = android_bundles.pop(0)

    android_bundles = np.array_split(android_bundles, 10)
    pool = Pool(10)
    func = partial(scrap)
    result = pool.map(func, android_bundles)
    pool.close()
    pool.join()
    features, bad = dict(), list()
    for i in result:
        features = dict(features, **i[0])
        bad = bad + i[1]
    # features, bad = scrap(list(reversed(android_bundles)))

    with open('features_ios_0time.pickle', 'wb') as f:
        pickle.dump(features, f)
    with open('bad_ios_0time.pickle', 'wb') as f:
        pickle.dump(bad, f)

    # print(features)
    # print(bad)

    # pool = Pool(5)
    # func = partial(scrap)
    # results = pool.map(func, [android_bundles[:20] for i in range(20)])
    # pool.close()
    # pool.join()
    #




good crown.match.masters.multiplayer.puzzle3d 0.0
good bubble.pop.shooter.panda.game 0.0
good free.coolsounds.rivex 0.0
good com.rainbowwallpaper.rainbowlive 0.0
good com.TwinSoftwareStudio.WarehouseSimulator2021 0.0
good com.phoneliving.talkingdroidfree 0.0
good com.osama.wallpapersbfffor2ss 0.0
good com.filaret.gotocardriving2 0.0
good com.studio.pham.hoshino.jewelsmaze 0.0
bad com.attacktitan.aotandroidapp 0.00012815583749839805
substring not found
good com.whitewind.MatchingPuzzle 0.00012815583749839805
good com.aromaticnectarineapps.meme 0.00012813941568426448
good com.linh.game.studio.jewelsadventure 0.00012813941568426448
good com.robots.arena 0.00012815583749839805
good com.bearboxstudio.cutegirlskin 0.00012815583749839805
good com.betterforsol.solitaire.card.games.free 0.00012815583749839805
good com.Three.Point.Contest.Basketball 0.00012813941568426448
good com.calendar.nokiaa 0.00012815583749839805
good io.voodoo.dune 0.00012815583749839805
good battletank.battlecity.tank199

good com.woodensword.zombie 0.0012815583749839804
good com.AppsBB.AvatarWallpapers4K 0.0016660258874791746
good com.outplayentertainment.booty_quest 0.0020504933999743688
good open.chat.whatsapp.directchat.message 0.0015376729882111738
good com.eralash.film 0.0017941817249775727
good com.xfungame.mrbullet 0.0019223375624759708
good com.supertapx.bigbigballer 0.0014097142124823785
good com.gos.amazing_police_rope_hero_chase 0.0017939518195797027
good com.branches.shooting.games 0.0017941817249775727
good com.fpgs.traintrack.train.station.construction.simulator.excavatorgames 0.002178649237472767
good com.grand.thief.plan.bankrobbery.simulator 0.0019223375624759708
good com.Playstal.Portal.Maze.Aperture.Jumper.Puzzle.Games.Teleport.Portal2 0.0016658124038954382
good net.lionbird.google.catEvolutionParty 0.0015378700499807767
good com.catmodcartoon.addoncatmodmaps 0.0019220912352639673
good com.appsorama.kleptocats 0.0017939518195797027
good com.city.coach.bus.driving.simulator.driver.par

good com.zombieshooting.survivalhunter.gun3d 0.003203485392106612
good com.mtg.heavyfarmingtractordriver 0.0037165192874535433
good com.cookapps.beagles.magic.cat.piano.tiles 0.0035883634499551454
good com.rooster.policeofficer 0.0035879036391594054
good com.teachchip.chipsbaldi.chipmods 0.0030757400999615533
good com.sablo.studios.house.mover 0.0038446751249519417
good com.HalfintheBoxTechnologies.WorldWarSlimeIdleDefence 0.0033316248077908763
good net.trikoder.android.kurir 0.0030753459764223477
good com.ggds.idle.car.factory.tycoon 0.0034602076124567475
good com.case7games.carparking 0.0032038959374599513
bad com.Fall.Guys_hwfree1 0.0038446751249519417
substring not found
good com.gdpplstck.plygrndsmltnrgdll 0.0037165192874535433
good com.beemoov.moonlight.ethan 0.00371604305484367
good com.ludia.tmnt 0.0032038959374599513
good com.tiles.hop.rush.fnf 0.00397283096245034
good com.livesoccertv 0.003459764223475141
good com.thekabirgames.virtual.dad.family.sim 0.003203485392106612
good

good com.dar.furniture.mods 0.00525438933743432
good com.sudoku.freemaster 0.005126233499935922
good com.rs.euro.truck.simulator.driving.game.free 0.005382545174932718
good com.yourelink.gifpro 0.005509994874423373
good com.xtreme.simcontruction.citybuilding.game2017 0.005126233499935922
good air.com.playcus.bingokingdomarena 0.00461361014994233
good com.boostcleanertools.app 0.004869297796002051
good com.vade.idlehome.minecraft.roblox.pixel.craft.build.maps 0.004998077662437524
good com.guncore.ultimateweapons 0.005382545174932718
good com.zebu.dietrecept 0.005253716043054844
good com.dreamplay.realmoto.google 0.005510701012431116
good com.KarasStudio.mod.CruisesInternationalShip 0.005638134290107637
bad org.nanobit.taboo.flexion 0.0057670126874279125
substring not found
good io.voodoo.tenkyu 0.004997437211686315
good com.istromgames.halloweencostumemakeupme 0.004741765987440728
good com.fgz.flying.ambulance.air.jet.transform.robot.games 0.005381855458739108
bad com.BiggsStore.XXXTent

good com.antarsoft.idlenumbers 0.006406970784213224
good com.dts.police.runner.car.cops.pursuit.games 0.006535947712418301
good com.funnyplaygames.realmotobikeracing 0.00743208610968734
good com.tikamori.guess.the.age 0.0070485710624118925
good com.lost.temple.run.temple.run.survival.game 0.006792259387415097
good com.ltg.games.to2048 0.007303946694003075
good com.pineconeapps.a4guessvideo 0.006279636037421504
good com.rtd.bulletfield.wargame.commandofreeshootinggames.eliteforcegame 0.007433038574907087
good com.ig.little.horror.nightamres 0.007176726899910291
good com.oldbutgold.cuttingtree 0.0065351101998974885
good it.Ettore.calcoliilluminotecnici 0.006664103549916698
good com.devuni.flashlight 0.007560225525371604
good ru.pdde.roadsigns 0.007176726899910291
good com.agilleapps.dlnaupnp 0.0064077918749199025
good com.bebidraw.drawtokyorevengers 0.007561194412405485
good com.wildlife.panther 0.007433038574907087
bad com.jeeda.fruity.helix.crush.helix.jump 0.0070485710624118925
substr

...

  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/site-packages/requests/sessions.py", line 533, in request
    resp = self.send(prep, **send_kwargs)
  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/site-packages/urllib3/connectionpool.py", line 344, in _make_request
    self._validate_conn(conn)
  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/site-packages/requests/sessions.py", line 646, in send
    r = adapter.send(request, **kwargs)
  File "<ipython-input-1-0c240dd9e60a>", line 116, in scrap
    response = requests.get(current_app_apkplz_url, timeout=10)
  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/site-packages/requests/api.py", line 75, in get
    return request('get', url, params=params, **kwargs)
  File "/

KeyboardInterrupt
  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/site-packages/requests/sessions.py", line 646, in send
    r = adapter.send(request, **kwargs)
  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/site-packages/urllib3/connectionpool.py", line 843, in _validate_conn
    conn.connect()
  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/ssl.py", line 689, in do_handshake
    self._sslobj.do_handshake()
  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/site-packages/urllib3/connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MA

KeyboardInterrupt: 

Process ForkPoolWorker-17:
Process ForkPoolWorker-16:
Process ForkPoolWorker-12:
Process ForkPoolWorker-14:
Process ForkPoolWorker-18:
Process ForkPoolWorker-20:
Process ForkPoolWorker-13:
Process ForkPoolWorker-11:
Process ForkPoolWorker-19:
Process ForkPoolWorker-15:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    sel

  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/shared/tools/anaconda3/envs/eburashnikov_python36/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


In [37]:
bundles["com.ivy.gems.witch"]

{'name': 'Gems Witch 1.1.6 APK',
 'price': ' Free ',
 'fileSize': ' 42.83',
 'rate': 4.4,
 'description': ' Gems Witch is the property and trademark from the developer Ivy. </p> <p> Gems Witch, gems match 3 adventure, new modern style in updated graphics, fun new game modes and a host of magic skills to help you crush through hundreds of levels! The newest game in jewels game or gem blast.<br/><br/>Selina the witch is learning how to brewed magical potion at home. And then an offer from magical school. A letter is in front of Selina. But an owl named Asio take her offer away. The story will continue from this... Help Selina grow up on this magical adventure.<br/><br/>Swipe, match, relax and crush gems, embark on magical adventure, and collect gold and enchanted staff on this one-of-a-kind enchanted exploration!<br/>Match and blast gems in fun gameplay and complete the mission goals.<br/>Become witch master in this match 3 puzzle game.<br/>Unlock castles or cities or various epic advent

===========================================

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pickle
import traceback
from multiprocessing import Pool
import os
import random
from functools import partial


In [2]:
def find_link_v2(html_str):
    pattern = 'data-placement="top" data-toggle="tooltip" title'
    id_start = html_str.index(pattern) + len(pattern)
    html_str = html_str[id_start:]
    soup = BeautifulSoup(html_str, "html.parser")
    link = str(soup.findAll('a')[0])
    
    pattern = '<a href="'
    id_start = link.index(pattern) + len(pattern)
    id_end = link[id_start:].index('">')
    link = link[id_start:id_start + id_end]

    return link
   

def find_name_v2(html_srt):
    pattern = 'class="product-header__title app-header__title">\n'
    id_start = html_srt.index(pattern) + len(pattern)
    id_end = html_srt[id_start:].index("\n")
    name = html_srt[id_start:id_start + id_end]
    return name


def find_price_v2(html_srt):
    pattern = '"price":'
    id_start = html_srt.index(pattern) + len(pattern)
    id_end = html_srt[id_start:].index(",")
    price = html_srt[id_start:id_start + id_end]
    return price


def find_genre_v2(html_srt):
    pattern = 'applicationCategory":"'
    id_start = html_srt.index(pattern) + len(pattern)
    id_end = html_srt[id_start:].index('"')
    genre = html_srt[id_start:id_start + id_end]
    return genre



def find_requierment_v2(html_srt):
    pattern = 'operatingSystem":"'
    id_start = html_srt.index(pattern) + len(pattern)
    id_end = html_srt[id_start:].index('"')
    requierments = html_srt[id_start:id_start + id_end]
    return requierments



def find_fileSize_v2(html_srt):
    pattern = 'Size</dt>\n<dd aria-label="'
    id_start = html_srt.index(pattern) + len(pattern)
    id_end = html_srt[id_start:].index('bytes')
    fileSize = html_srt[id_start:id_start + id_end]
    return fileSize



def find_rate_v2(html_str):
    pattern = 'ratingValue":'
    id_start = html_str.index(pattern) + len(pattern)
    id_end = html_str[id_start:].index(',')
    rate = float(html_str[id_start:id_start + id_end])
    html_str = html_str[id_start:]
    pattern = 'reviewCount":'
    id_start = html_str.index(pattern) + len(pattern)
    id_end = html_str[id_start:].index('}')
    rewiew = int(html_str[id_start:id_start + id_end])
    if rewiew > 10:
        return rate
    else:
        return 0

    
def find_description_v2(html_srt):
    pattern = 'test-bidi="" dir="false">'
    id_start = html_srt.index(pattern) + len(pattern)
    id_end = html_srt[id_start:].index('</p>')
    description = html_srt[id_start:id_start + id_end]
    return description


def scrap_ios_v2(ios_bundles):
    url_appfollow_play = 'https://apps.apple.com/us/app/id'
    #     url_apkplz = "https://apkplz.net/app/"
    #     url_q_apkplz = "https://apkplz.net/search?q="
    bad_bundles = []
    features = {}

    counter = 1
    save_id = 0

    while counter < len(ios_bundles):
        try:
            current_bundle = ios_bundles[counter]
#             time.sleep(0.5)

            current_app_apkplz_url = url_appfollow_play + current_bundle
            response = requests.get(current_app_apkplz_url, timeout=10)
            soup = BeautifulSoup(response.text, "html.parser")
            str_soup = str(soup)
    #         try:
    #             link = find_link(str_soup)
    #         except:
    #             print("bad", current_bundle, counter / len(ios_bundles))
    #             bad_bundles.append(current_bundle)
    #             counter = counter + 1
    #             continue 

            try:
                name = find_name_v2(str_soup)
            except:
                name = np.nan
            try:
                price = find_price_v2(str_soup)
            except:
                price = np.nan
            try:
                requierments = find_requierment_v2(str_soup)
            except:
                requierments = np.nan
            try:
                fileSize = find_fileSize_v2(str_soup)
            except:
                fileSize = np.nan
            try:
                rate = find_rate_v2(str_soup)
            except:
                rate = np.nan
            try:
                description = find_description_v2(str_soup)
            except:
                description = np.nan
            try:
                genre = find_genre_v2(str_soup)
            except:
                genre = np.nan

            features[current_bundle] = {}
            features[current_bundle]["name"] = name
            features[current_bundle]["price"] = price
            features[current_bundle]["genre"] = genre

            features[current_bundle]["fileSize"] = fileSize
            features[current_bundle]["rate"] = rate
            features[current_bundle]["description"] = description
            features[current_bundle]["requierments"] = requierments

    #         if counter == 30:
    #             break
            counter = counter + 1
            print("good", current_bundle, counter / len(ios_bundles))
        except:
            counter = counter + 1
            print("bad", current_bundle, counter / len(ios_bundles))

    return features, bad_bundles


In [10]:
if __name__ == "__main__":

    with open('bundles.pickle', 'rb') as f:
        bundles = pickle.load(f)

    ios_bundles = list()
    android_bundles = list()

    for i in bundles:
        try:
            tmp = int(i)
            ios_bundles.append(i)
        except:
            android_bundles.append(i)
    drop_nan = android_bundles.pop(0)
    ios_bundles = np.array_split(ios_bundles, 20)
    for j, ib in enumerate(ios_bundles):
        
#         features, bad_bundles = scrap_ios_v2(ib)
#         time.sleep(30)
        ib = np.array_split(ib, 1)

        pool = Pool(1)
        func = partial(scrap_ios_v2)
        result = pool.map(func, ib)
        pool.close()
        pool.join()
        features, bad = dict(), list()
        for i in result:
            features = dict(features, **i[0])
            bad = bad + i[1]
        # features, bad = scrap(list(reversed(android_bundles)))

        with open('features_ios_'+str(j)+'.pickle', 'wb') as f:
            pickle.dump(features, f)
        with open('bad_ios_'+str(j)+'.pickle', 'wb') as f:
            pickle.dump(bad, f)
        break


good 1356045010 0.0030581039755351682
good 675586452 0.0045871559633027525
good 1338700502 0.0061162079510703364
good 1483222663 0.00764525993883792
good 1488312219 0.009174311926605505
good 1351283922 0.010703363914373088
good 1078203264 0.012232415902140673
good 1369836531 0.013761467889908258
good 1538914898 0.01529051987767584
good 979324502 0.016819571865443424
good 1448992461 0.01834862385321101
good 1443446174 0.019877675840978593
good 507738236 0.021406727828746176
good 923653602 0.022935779816513763
good 1576740922 0.024464831804281346
good 1126772151 0.02599388379204893
good 1565768885 0.027522935779816515
good 1503462284 0.0290519877675841
good 504539730 0.03058103975535168
good 1332998468 0.03211009174311927
good 1574344057 0.03363914373088685
good 1575332738 0.035168195718654434
good 1563892945 0.03669724770642202
good 1548113050 0.0382262996941896
good 1500853741 0.039755351681957186
good 1164803453 0.04128440366972477
good 1549766306 0.04281345565749235
good 956736147 0.

good 1544913085 0.3547400611620795
good 1504500795 0.3562691131498471
good 1572251659 0.3577981651376147
good 1534137838 0.35932721712538224
good 1081720194 0.36085626911314983
good 1522823359 0.3623853211009174
good 1017551780 0.363914373088685
good 798939657 0.3654434250764526
good 1448250507 0.3669724770642202
good 1530143270 0.36850152905198774
good 1253027907 0.37003058103975534
good 1477975397 0.37155963302752293
good 929195587 0.3730886850152905
good 1557220713 0.3746177370030581
good 1498421999 0.3761467889908257
good 1503651159 0.37767584097859325
good 1142617026 0.37920489296636084
good 1408624516 0.38073394495412843
good 1559490250 0.382262996941896
good 1478379474 0.3837920489296636
good 1473407568 0.3853211009174312
good 1564919400 0.38685015290519875
good 1406725978 0.38837920489296635
good 499125840 0.38990825688073394
good 1487412155 0.39143730886850153
good 1143489052 0.3929663608562691
good 1451817911 0.3944954128440367
good 1465222322 0.39602446483180426
good 1463349

good 1556070521 0.7140672782874617
good 1235044718 0.7155963302752294
good 1553246659 0.7171253822629969
good 1546343654 0.7186544342507645
good 1504892791 0.7201834862385321
good 1562874459 0.7217125382262997
good 1376579740 0.7232415902140673
good 1565303313 0.7247706422018348
good 1495581077 0.7262996941896025
good 973925909 0.72782874617737
good 1484976354 0.7293577981651376
good 1391720843 0.7308868501529052
good 1474907432 0.7324159021406728
good 649260774 0.7339449541284404
good 1525724626 0.735474006116208
good 1525990438 0.7370030581039755
good 563494175 0.7385321100917431
good 1564391515 0.7400611620795107
good 1468246255 0.7415902140672783
good 939480693 0.7431192660550459
good 891616303 0.7446483180428135
good 1336007559 0.746177370030581
good 1518272534 0.7477064220183486
good 1511924085 0.7492354740061162
good 505658864 0.7507645259938838
good 1357942090 0.7522935779816514
good 1511168571 0.753822629969419
good 1443058056 0.7553516819571865
good 1239065985 0.7568807339449

In [125]:
with open('features_ios.pickle', 'rb') as f:
    features_ios = pickle.load(f)
features_ios

{'633246396': {'name': '          Brain Training, Know brain age',
  'price': '0',
  'genre': 'Games',
  'fileSize': '33.5 mega',
  'rate': 4.3,
  'description': 'Find out your brain age.It just only 30 sec per games.<br/>Simple yet addicting app which tests your brain age base on your game performance.<br/><br/>● Observation<br/>Tap the numbers 1 to 25<br/>● Memory<br/>Memory alignment of numbers from 0 to 9<br/>● Judgement<br/>Select the "Text color"<br/>● Intuition<br/>Tap the specified number of seconds<br/>● Computing<br/>Elementary-level math calculations<br/>● Reaction<br/>Rock paper scissors by order<br/><br/>● Today\'s puzzle game<br/>Funny puzzle game every day.<br/>[Mini Sudoku], [Cross Puzzle], [CALC]<br/><br/>Try General Test detect your brain age.',
  'requierments': 'Requires iOS\xa09.0 or later. Compatible with iPhone, iPad, and iPod\xa0touch.'},
 '1434219392': {'name': '          Zodi: Horoscope &amp; Astrology',
  'price': '0',
  'genre': 'Lifestyle',
  'fileSize': '1